EXPORTING DATASET TO AWS DYNAMODB

In [ ]:
import pandas as pd
import boto3

# DynamoDB table

# Dataset definition
original_dataset = pd.read_csv('data/hotel_original.csv')

# VAR AWS Region
region_name = 'us-east-1'

# VAR AWS DynamoDB Table
dynamodb_table_name = ''  

# Send to DynamoDb
dynamodb = boto3.resource('dynamodb', region_name)

items = []
for index, row in original_dataset.iterrows():
    item = {
        'Booking_ID': {'S': (row['Booking_ID'])},
        'no_of_adults': {'N': str(row['no_of_adults'])},
        'no_of_children': {'N': str(row['no_of_children'])},
        'no_of_weekend_nights': {'N': str(row['no_of_weekend_nights'])},
        'no_of_week_nights': {'N': str(row['no_of_week_nights'])},
        'type_of_meal_plan': {'S': row['type_of_meal_plan']},
        'required_car_parking_space': {'N': str(row['required_car_parking_space'])},
        'room_type_reserved': {'S': row['room_type_reserved']},
        'lead_time': {'N': str(row['lead_time'])},
        'arrival_year': {'N': str(row['arrival_year'])},
        'arrival_month': {'N': str(row['arrival_month'])},
        'arrival_date': {'N': str(row['arrival_date'])},
        'market_segment_type': {'S': row['market_segment_type']},
        'repeated_guest': {'N': str(row['repeated_guest'])},
        'no_of_previous_cancellations': {'N': str(row['no_of_previous_cancellations'])},
        'no_of_previous_bookings_not_canceled': {'N': str(row['no_of_previous_bookings_not_canceled'])},
        'avg_price_per_room': {'N': str(row['avg_price_per_room'])},
        'no_of_special_requests': {'N': str(row['no_of_special_requests'])},
        'booking_status': {'S': row['booking_status']}
    }
    items.append(item)

dynamodb = boto3.client('dynamodb', region_name)
for item in items:
    response = dynamodb.put_item(
        TableName=dynamodb_table_name,
        Item=item
    )

IMPORTING DATASET FROM DYNAMODB